Upload .mat file into Pithon and see its structure | Загрузить .mat файл в Python и посмотреть его структуру

In [ ]:
import numpy as np
from scipy.io import loadmat

data = loadmat('ex4data1.mat')
data

Let's check the dimensionality of the Data Frame | Давайте определим размерность загруженных данных 'X' и 'y'

In [ ]:
import pandas as pd
data['X'].shape, data['y'].shape

Let's have a look into any array of 'X' | Давайте посмотрим на структуру любой строки в массиве 'X', например строки 4999  

In [ ]:
data['X'][4999]